In [1]:
# === Standard Library ===
import math
import random
import time
import heapq
import itertools
from collections import defaultdict, deque
from itertools import combinations
from typing import Any, Tuple, Dict, List, Set, Sequence, Union

# === Third-Party Libraries ===

# --- Scientific Computing ---
import numpy as np
import pandas as pd
import scipy.sparse as sp
from scipy.optimize import linprog

# --- Plotting ---
import matplotlib.pyplot as plt

# --- Parallel Processing ---
from joblib import Parallel, delayed
from tqdm import tqdm

# --- Graph Processing ---
import networkx as nx

# --- JIT Compilation ---
from numba import njit, prange

In [2]:
def nx_to_csr(G: nx.Graph) -> Tuple[List[int], Dict[int, int], np.ndarray, np.ndarray, np.ndarray]:
     """Convert an undirected NetworkX graph (edge attr `'p'`) to CSR arrays."""
     nodes: List[int] = list(G.nodes())
     idx_of: Dict[int, int] = {u: i for i, u in enumerate(nodes)}

     indptr: List[int] = [0]
     indices: List[int] = []
     probs: List[float] = []

     for u in nodes:
         for v in G.neighbors(u):
             indices.append(idx_of[v])
             probs.append(G.edges[u, v]['p'])
         indptr.append(len(indices))

     return (
         nodes,
         idx_of,
         np.asarray(indptr, dtype=np.int32),
         np.asarray(indices, dtype=np.int32),
         np.asarray(probs, dtype=np.float32),
     )

@njit(inline="always")
def _bfs_component_size(start: int,
                    indptr: np.ndarray,
                    indices: np.ndarray,
                    probs: np.ndarray,
                    deleted: np.ndarray) -> int:
    """Return |C_u|−1 for **one** random realisation (stack BFS)."""
    n = deleted.size
    stack = np.empty(n, dtype=np.int32)
    visited = np.zeros(n, dtype=np.uint8)

    size = 1
    top = 0
    stack[top] = start
    top += 1
    visited[start] = 1

    while top:
        top -= 1
        v = stack[top]
        for eid in range(indptr[v], indptr[v + 1]):
            w = indices[eid]
            if deleted[w]:
                continue
            if np.random.random() >= probs[eid]:
                continue
            if visited[w]:
                continue
            visited[w] = 1
            stack[top] = w
            top += 1
            size += 1
    return size - 1

@njit(parallel=True)
def epc_mc(indptr: np.ndarray,
            indices: np.ndarray,
            probs: np.ndarray,
            deleted: np.ndarray,
            num_samples: int) -> float:
    """Monte‑Carlo estimator of **expected pairwise connectivity** (EPC)."""
    surv = np.where(~deleted)[0]
    m = surv.size
    if m < 2:
        return 0.0

    acc = 0.0
    for _ in prange(num_samples):
        u = surv[np.random.randint(m)]
        acc += _bfs_component_size(u, indptr, indices, probs, deleted)

    return (m * acc) / (2.0 * num_samples)

def epc_mc_deleted(
  G: nx.Graph,
  S: set,
  num_samples: int = 100_000,
) -> float:
  # build csr once
  nodes, idx_of, indptr, indices, probs = nx_to_csr(G)
  n = len(nodes)

  # turn python set S into a mask (node-IDs to delete)
  deleted = np.zeros(n, dtype=np.bool_)
  for u in S:
    deleted[idx_of[u]] = True

  epc = epc_mc(indptr, indices, probs, deleted, num_samples)

  return epc

In [3]:
def grasp_cndp(G: nx.Graph,
               K: int,
               alpha: float = 0.1,
               num_samples: int = 1000,
               restarts: int = 15):
    """
    GRASP for Stochastic CNDP:
    """
    best_S, best_score = None, float('inf')

    for _ in range(restarts):
        S = set()
        # precompute σ(empty)
        sigma_S = epc_mc_deleted(G, S, num_samples)

        for k in range(K):
            # 1) compute improvement d_j = σ(S) – σ(S ∪ {j})
            improvements = {}
            for j in G.nodes():
                if j in S: 
                    continue
                sigma_Sj = epc_mc_deleted(G, S | {j}, num_samples)
                improvements[j] = sigma_S - sigma_Sj

            # 2) find best and worst d
            max_imp = max(improvements.values())
            min_imp = min(improvements.values())

            # 3) build RCL = { j : d_j ≥ max_imp – alpha*(max_imp – min_imp) }
            threshold = max_imp - alpha * (max_imp - min_imp)
            RCL = [j for j, d in improvements.items() if d >= threshold]

            # 4) pick one at random from RCL
            v = random.choice(RCL)
            S.add(v)

            # 5) update σ(S)
            sigma_S = epc_mc_deleted(G, S, num_samples)

        # end of one construction
        if sigma_S < best_score:
            best_score = sigma_S
            best_S = S.copy()

    return best_S, best_score

In [ ]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0
K = 10
alpha = 0.4

# assign uniform edge‐survival 0.8
for u, v in G.edges():
  G[u][v]['p'] = p_edge

# for alpha in np.arange(0.1, 1.1, 0.1):
S_star, score_star = grasp_cndp(G, K=K,
                                alpha=alpha,
                                num_samples=10_000,
                                restarts=20)
print("Best removal set:", S_star)
print("Estimated σ(S*)  :", score_star)

100%|██████████| 20/20 [01:17<00:00,  3.88s/it]


Best removal set: {64, 0, 6, 27, 43, 47, 79, 50, 59, 94}
Estimated σ(S*)  : 3031.1325


100%|██████████| 20/20 [01:17<00:00,  3.87s/it]


Best removal set: {0, 64, 65, 36, 43, 47, 17, 50, 59, 94}
Estimated σ(S*)  : 2975.3595


100%|██████████| 20/20 [01:18<00:00,  3.93s/it]


...

Best removal set: {64, 5, 37, 6, 69, 70, 45, 24, 94, 30}
Estimated σ(S*)  : 3476.484


100%|██████████| 20/20 [01:22<00:00,  4.12s/it]


Best removal set: {96, 0, 47, 49, 19, 83, 86, 55, 90, 62}
Estimated σ(S*)  : 3555.2115


  0%|          | 0/20 [00:02<?, ?it/s]


KeyboardInterrupt: 

In [4]:
def local_search_swap(
    S: Set[int],
    *,
    csr: Tuple[List[int], Dict[int,int], np.ndarray, np.ndarray, np.ndarray],
    num_samples: int = 100000,
    max_iter: int = 5
) -> Set[int]:
    """
    Given initial delete-set S, try 1-for-1 swaps to reduce EPC.
    csr = (nodes, idx_of, indptr, indices, probs).
    """
    nodes, idx_of, indptr, indices, probs = csr
    n = len(nodes)
    deleted = np.zeros(n, dtype=bool)
    for u in S:
        deleted[idx_of[u]] = True

    curr = epc_mc(indptr, indices, probs, deleted, num_samples)

    for _ in range(max_iter):
        best_delta = 0.0
        best_swap = None

        for i in list(S):
            ii = idx_of[i]
            deleted[ii] = False
            for j, jj in idx_of.items():
                if deleted[jj]:
                    continue
                deleted[jj] = True
                sigma_new = epc_mc(indptr, indices, probs, deleted, num_samples)
                delta = curr - sigma_new
                if delta > best_delta:
                    best_delta = delta
                    best_swap = (ii, jj, sigma_new)
                deleted[jj] = False
            deleted[ii] = True

        if best_swap is None:
            break

        ii, jj, curr = best_swap
        deleted[ii] = False
        deleted[jj] = True
        S.remove(nodes[ii])
        S.add(nodes[jj])

    return S

In [5]:
def grasp_with_local_search(
    G: nx.Graph,
    K: int,
    alpha: float = 0.2,
    mc_samples_grasp: int = 10000,
    mc_samples_ls: int = 10000,
    restarts: int = 20,
    max_ls_iter: int = 5
) -> Tuple[Set[int], float]:
    """
    Combined GRASP + local_search_swap procedure.
    """
    def build_csr(G: nx.Graph):

        nodes = sorted(G.nodes())
        idx_of = {u: i for i, u in enumerate(nodes)}
        n = len(nodes)

        degs = [len(list(G.neighbors(u))) for u in nodes]
        indptr = np.zeros(n + 1, dtype=int)
        indptr[1:] = np.cumsum(degs)

        indices = np.empty(indptr[-1], dtype=int)
        probs = np.empty(indptr[-1], dtype=float)
        ptr = 0

        for u in nodes:
            for v in G.neighbors(u):
                indices[ptr] = idx_of[v]
                probs[ptr] = G.edges[u, v]['p']
                ptr += 1
                
        return nodes, idx_of, indptr, indices, probs

    csr = build_csr(G)
    best_S, best_score = set(), float('inf')

    for _ in tqdm(range(restarts)):
        S, _ = grasp_cndp(
            G, K, num_samples=mc_samples_grasp, 
            alpha=alpha, restarts=5)
        
        S_imp = local_search_swap(
            S, csr=csr, num_samples=mc_samples_ls, 
            max_iter=max_ls_iter)
        
        score_imp = epc_mc_deleted(
            G, S_imp, 
            num_samples=mc_samples_grasp)
        
        if score_imp < best_score:
            best_score, best_S = score_imp, S_imp.copy()

    return best_S, best_score

In [6]:
G = nx.erdos_renyi_graph(100, 0.0443, 42)
p_edge = 1.0
K = 10
alpha = 0.4

# assign uniform edge‐survival 0.8
for u, v in G.edges():
  G[u][v]['p'] = p_edge


S_star, score_star = grasp_with_local_search(
                                G, K=K,
                                alpha=alpha,
                                mc_samples_grasp=10_000,
                                mc_samples_ls=10_000,
                                restarts=10,
                                max_ls_iter=5)

print("Best removal set:", S_star)
print("Estimated σ(S*)  :", score_star)

100%|██████████| 10/10 [15:03<00:00, 90.39s/it]

Best removal set: {0, 65, 34, 43, 79, 50, 24, 27, 93, 94}
Estimated σ(S*)  : 2853.333


In [9]:
class ReactiveAlpha:
    def __init__(self, alpha_vals: List[float]):
        self.alpha_vals = alpha_vals
        self.weights = [1.0] * len(alpha_vals)

    def sample(self) -> Tuple[int, float]:
        total = sum(self.weights)
        r = random.random() * total
        cum = 0.0
        for i, w in enumerate(self.weights):
            cum += w
            if r <= cum:
                return i, self.alpha_vals[i]
        return len(self.weights)-1, self.alpha_vals[-1]

    def reward(self, idx: int, amount: float = 1.0):
        self.weights[idx] += amount

    def penalize(self, idx: int, factor: float = 0.99):
        self.weights[idx] *= factor

def grasp_construct(G: nx.Graph,
                    K: int,
                    alpha: float,
                    mc_samples: int) -> Tuple[Set[int], float]:
    """One GRASP construction (no restarts)."""
    S: Set[int] = set()
    cache: Dict[frozenset, float] = {}

    def sigma(SetS: Set[int]) -> float:
        key = frozenset(SetS)
        if key not in cache:
            cache[key] = epc_mc_deleted(G, SetS, num_samples=mc_samples)
        return cache[key]

    sigma_S = sigma(S)
    for _ in range(K):
        gains = {}
        for v in G.nodes():
            if v in S:
                continue
            gains[v] = sigma_S - sigma(S | {v})
        d_max, d_min = max(gains.values()), min(gains.values())
        thresh = d_max - alpha * (d_max - d_min)
        RCL = [v for v, d in gains.items() if d >= thresh]
        choice = random.choice(RCL)
        S.add(choice)
        sigma_S = sigma(S)

    return S, sigma_S

def local_search_swap(
    S: Set[int],
    *,
    csr: Tuple[List[int], Dict[int,int], np.ndarray, np.ndarray, np.ndarray],
    num_samples: int = 100_000,
    max_iter: int = 5
) -> Set[int]:
    nodes, idx_of, indptr, indices, probs = csr
    n = len(nodes)
    deleted = np.zeros(n, dtype=bool)
    for u in S:
        deleted[idx_of[u]] = True

    curr = epc_mc(indptr, indices, probs, deleted, num_samples)

    for _ in range(max_iter):
        best_delta = 0.0
        best_swap = None
        for i in list(S):
            ii = idx_of[i]
            deleted[ii] = False
            for j in nodes:
                jj = idx_of[j]
                if deleted[jj]:
                    continue
                deleted[jj] = True
                new_sigma = epc_mc(indptr, indices, probs, deleted, num_samples)
                delta = curr - new_sigma
                if delta > best_delta:
                    best_delta = delta
                    best_swap = (ii, jj, new_sigma)
                deleted[jj] = False
            deleted[ii] = True

        if best_swap is None:
            break
        ii, jj, curr = best_swap
        deleted[ii] = False
        deleted[jj] = True
        S.remove(nodes[ii])
        S.add(nodes[jj])

    return S

def path_relink(S: Set[int], E: Set[int],
                G: nx.Graph,
                mc_samples: int) -> Tuple[Set[int], float]:
    """Greedy walk from S toward E, returning best intermediate."""

    T = S.copy()
    best_T, best_score = T.copy(), epc_mc_deleted(G, T, mc_samples)
    D_add = list(E - T)
    D_rm  = list(T - E)

    while D_add and D_rm:
        best_move = None
        best_delta = 0.0
        for i in D_rm:
            for j in D_add:
                T_candidate = T.copy()
                T_candidate.remove(i)
                T_candidate.add(j)
                score = epc_mc_deleted(G, T_candidate, mc_samples)
                delta = best_score - score
                if delta > best_delta:
                    best_delta = delta
                    best_move = (i, j, score)
        if best_move is None:
            break
        i, j, new_score = best_move
        T.remove(i); T.add(j)
        D_rm.remove(i); D_add.remove(j)
        if new_score < best_score:
            best_score = new_score
            best_T = T.copy()

    return best_T, best_score


def insert_into_elite(elite: List[Tuple[Set[int], float]],
                      candidate: Tuple[Set[int], float],
                      max_size: int = 10):
    elite.append(candidate)
    elite.sort(key=lambda x: x[1])
    if len(elite) > max_size:
        elite.pop()

def build_csr(G: nx.Graph):
    nodes = sorted(G.nodes())
    idx_of = {u: i for i, u in enumerate(nodes)}
    degs = [len(list(G.neighbors(u))) for u in nodes]
    indptr = np.zeros(len(nodes)+1, dtype=int)
    indptr[1:] = np.cumsum(degs)
    indices = np.empty(indptr[-1], dtype=int)
    probs   = np.empty(indptr[-1], dtype=float)
    ptr = 0
    for u in nodes:
        for v in G.neighbors(u):
            indices[ptr] = idx_of[v]
            probs[ptr]   = G.edges[u, v]['p']
            ptr += 1
    return nodes, idx_of, indptr, indices, probs

def grasp_meta(
    G: nx.Graph,
    K: int,
    restarts: int = 50,
    grasp_restarts: int = 20,
    mc_samples_grasp: int = 2000,
    mc_samples_ls: int = 10000,
    mc_samples_pr: int = 5000,
    max_ls_iter: int = 3,
    elite_size: int = 5
) -> Tuple[Set[int], float]:
    
    reactive = ReactiveAlpha([0.05, 0.15, 0.3, 0.5, 0.7, 0.9])
    csr = build_csr(G)

    elite: List[Tuple[Set[int], float]] = []

    for _ in tqdm(range(restarts), desc="Processing grasp meta",
                  total=int(restarts)):
        
        # 1) pick alpha
        idxα, alpha = reactive.sample()

        # 2) GRASP construction
        # grasp with no restarts
        S0, score0 = grasp_construct(G, K, alpha, mc_samples_grasp)

        # S0, score0 = grasp_cndp(G, K, num_samples=mc_samples_grasp,
        #                         alpha=alpha, restarts=grasp_restarts)

        # 3) local search polishing
        S1 = local_search_swap(
            S0, csr=csr, num_samples=mc_samples_ls, max_iter=max_ls_iter)
        score1 = epc_mc_deleted(G, S1, num_samples=mc_samples_grasp)

        # 4) path relinking against each elite
        for E, sE in elite:
            Spr, spr_score = path_relink(S1, E, G, mc_samples_pr)
            if spr_score < score1:
                S1, score1 = Spr, spr_score

        # 5) update elite and reactive α
        insert_into_elite(elite, (S1, score1), max_size=elite_size)

        quantiles = [s for _, s in elite]
        
        if score1 <= quantiles[max(1, len(quantiles)//10) - 1]:
            reactive.reward(idxα)
        else:
            reactive.penalize(idxα)

    # return best from elite
    best_S, best_score = min(elite, key=lambda x: x[1])
    return best_S, best_score

In [10]:
NODES = 100
p = 0.0443
SEED = 42

H0 = nx.erdos_renyi_graph(NODES, p, seed=SEED)

K = 10
alpha = 0.4
p_edge = 1.0

for u, v in H0.edges():
    H0[u][v]['p'] = p_edge

best_s, best_epc = grasp_meta(H0, K, mc_samples_grasp = 10_000,
                              mc_samples_ls=10_000, mc_samples_pr=10_000,
                              max_ls_iter=5, elite_size=5)

print(f"Best removal set: {best_s}")
print(f"Estimated epc(Best_S): {best_epc}")

Processing grasp meta: 100%|██████████| 50/50 [34:19<00:00, 41.18s/it]

Best removal set: {0, 65, 34, 59, 79, 50, 24, 27, 93, 94}
Estimated epc(Best_S): 2825.055
